In [ ]:
#!pip install tqdm
#!pip install opencv-python

# Libraries

In [1]:
from __future__ import absolute_import
from __future__ import print_function

import keras
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

Using TensorFlow backend.


In [2]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

In [3]:
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

# Generate Training Data

```
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar -O /tmp/VOCtrainval_11-May-2012.tar
!tar xf /tmp/VOCtrainval_11-May-2012.tar -C /tmp/
!rm -rf /tmp/VOCtrainval_11-May-2012.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar -O /tmp/VOCtrainval_06-Nov-2007.tar
!tar xf /tmp/VOCtrainval_06-Nov-2007.tar -C /tmp/
!rm -rf /tmp/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar -O /tmp/VOCtest_06-Nov-2007.tar
!tar xf /tmp/VOCtest_06-Nov-2007.tar -C /tmp/
!rm -rf /tmp/VOCtest_06-Nov-2007.tar
```

```
--2018-10-19 21:39:14--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘/tmp/VOCtrainval_11-May-2012.tar’

/tmp/VOCtrainval_11 100%[===================>]   1.86G  7.45MB/s    in 4m 6s   

2018-10-19 21:43:20 (7.76 MB/s) - ‘/tmp/VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]

--2018-10-19 21:43:23--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘/tmp/VOCtrainval_06-Nov-2007.tar’

/tmp/VOCtrainval_06 100%[===================>] 438.72M  8.17MB/s    in 60s     

2018-10-19 21:44:24 (7.27 MB/s) - ‘/tmp/VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

--2018-10-19 21:44:25--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘/tmp/VOCtest_06-Nov-2007.tar’

/tmp/VOCtest_06-Nov 100%[===================>] 430.13M  8.06MB/s    in 56s     

2018-10-19 21:45:22 (7.63 MB/s) - ‘/tmp/VOCtest_06-Nov-2007.tar’ saved [451020800/451020800]
```

In [ ]:
# Default classes for VOCdevkit
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

# Initialize the Batch generator for datasets
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

# The directories that contain the images.
VOC_2007_images_dir      = '/tmp/VOCdevkit/VOC2007/JPEGImages/'
VOC_2012_images_dir      = '/tmp/VOCdevkit/VOC2012/JPEGImages/'

# The directories that contain the annotations.
VOC_2007_annotations_dir      = '/tmp/VOCdevkit/VOC2007/Annotations/'
VOC_2012_annotations_dir      = '/tmp/VOCdevkit/VOC2012/Annotations/'

# The paths to the image sets.
VOC_2007_train_image_set_filename    = '/tmp/VOCdevkit/VOC2007/ImageSets/Main/train.txt'
VOC_2012_train_image_set_filename    = '/tmp/VOCdevkit/VOC2012/ImageSets/Main/train.txt'
VOC_2007_val_image_set_filename      = '/tmp/VOCdevkit/VOC2007/ImageSets/Main/val.txt'
VOC_2012_val_image_set_filename      = '/tmp/VOCdevkit/VOC2012/ImageSets/Main/val.txt'
VOC_2007_trainval_image_set_filename = '/tmp/VOCdevkit/VOC2007/ImageSets/Main/trainval.txt'
VOC_2012_trainval_image_set_filename = '/tmp/VOCdevkit/VOC2012/ImageSets/Main/trainval.txt'
VOC_2007_test_image_set_filename     = '/tmp/VOCdevkit/VOC2007/ImageSets/Main/test.txt'

# Create Training Dataset
train_dataset.parse_xml(images_dirs=[VOC_2007_images_dir,
                                     VOC_2012_images_dir],
                        image_set_filenames=[VOC_2007_trainval_image_set_filename,
                                             VOC_2012_trainval_image_set_filename],
                        annotations_dirs=[VOC_2007_annotations_dir,
                                          VOC_2012_annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)

# Create Testing Dataset
val_dataset.parse_xml(images_dirs=[VOC_2007_images_dir],
                      image_set_filenames=[VOC_2007_test_image_set_filename],
                      annotations_dirs=[VOC_2007_annotations_dir],
                      classes=classes,
                      include_classes='all',
                      exclude_truncated=False,
                      exclude_difficult=True,
                      ret=False)

# Convert the dataset into an HDF5 dataset fpor S3 upload
train_dataset.create_hdf5_dataset(file_path='/tmp/train.h5',
                                  resize=False,
                                  variable_image_size=True,
                                  verbose=True)

val_dataset.create_hdf5_dataset(file_path='/tmp/test.h5',
                                resize=False,
                                variable_image_size=True,
                                verbose=True)

In [4]:
prefix = 'data'
channels = {'train': 's3://{}'.format(bucket)}
#sagemaker_session.upload_data(path='/tmp/train.h5', bucket=bucket, key_prefix=prefix)
#sagemaker_session.upload_data(path='/tmp/test.h5', bucket=bucket, key_prefix=prefix)
hyperparameters = dict(learning_rate=.001, epochs=30)
output_location = 's3://{}/output'.format(bucket)
image_name = '500842391574.dkr.ecr.us-west-2.amazonaws.com/keras:keras-gpu'

# Train

In [5]:
BYOC_estimator = Estimator(
    image_name,
    role=role,
    output_path=output_location,
    train_instance_count=1,
    train_instance_type='ml.p3.8xlarge',
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)

In [ ]:
BYOC_estimator.fit(channels)

INFO:sagemaker:Creating training-job with name: keras-2018-10-19-23-15-25-366


2018-10-19 23:15:25 Starting - Starting the training job...
2018-10-19 23:15:26 Starting - Launching requested ML instances......